In [20]:
import gc
from pathlib import Path

import numpy as np
import cupy as cp
import cudf
import xarray as xr
import dask
import dask.array as da
import dask_cudf

from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

from cuml.dask.common.utils import persist_across_workers
from cuml.dask.ensemble import RandomForestClassifier as DaskRandomForest
from sklearn.metrics import classification_report, confusion_matrix
from cuml.metrics import accuracy_score

from sklearn.model_selection import train_test_split

In [2]:
# %%
cluster = LocalCUDACluster(
    threads_per_worker=1,
    local_directory="/raid/jjayabaskar/dask-tmp",
    CUDA_VISIBLE_DEVICES=[0,1,2,3,4,5,6,7],
)
client = Client(cluster)
client


2025-11-19 17:04:04,897 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/tornado/websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/tornado/web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is e

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 503.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38323,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34005,Total threads: 1
Dashboard: http://127.0.0.1:39821/status,Memory: 62.97 GiB
Nanny: tcp://127.0.0.1:44791,


2025-11-19 17:04:13,206 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/tornado/websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/tornado/web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/raid/jjayabaskar/gis12/lib/python3.12/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired. Configure the app with a larger value for --session-token-expiration if necessary")
bokeh.protocol.exceptions.ProtocolError: Token is e

In [3]:

workers = client.has_what().keys()
n_workers = len(workers)
n_workers

8

In [4]:
features_store = Path("/raid/jjayabaskar/outputs/sentinel_features/sentinel2_2022_annual.zarr")
labels_store = Path("/raid/jjayabaskar/outputs/sentinel_features/worldcover_2022_annual.zarr")
predictions_store = Path("/raid/jjayabaskar/outputs/sentinel_features/worldcover_predictions_2022.zarr")

target_year = 2022
train_fraction = 0.8
sample_fraction = 1.0
flatten_chunk_rows = 1_000_000
random_state = 42

worldcover_classes = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland",
}
nodata_value = 0
valid_classes = list(worldcover_classes.keys())

In [5]:
feature_ds = xr.open_zarr(features_store, consolidated=True, chunks="auto").unify_chunks()
label_ds = xr.open_zarr(labels_store, consolidated=True, chunks="auto").unify_chunks()

spectral = feature_ds["bands"].sel(year=target_year)
spectral = spectral.assign_coords(band=[str(b) for b in spectral.band.values])

ndvi = feature_ds["NDVI"].sel(year=target_year).expand_dims(band=["NDVI"])
ndwi = feature_ds["NDWI"].sel(year=target_year).expand_dims(band=["NDWI"])

features = xr.concat([spectral, ndvi, ndwi], dim="band")
labels = label_ds["worldcover"].sel(year=target_year)

band_names = list(features.band.values)
band_names

['B02', 'B03', 'B04', 'B08', 'NDVI', 'NDWI']

In [6]:
valid_mask = (
    da.isfinite(features.data).all(axis=0)
    & da.isfinite(labels.data)
    & (labels.data != nodata_value)
    & da.isin(labels.data, valid_classes)
)

features = features.where(valid_mask)
labels = labels.where(valid_mask)

In [7]:
stacked_features = features.stack(sample=("y", "x")).transpose("sample", "band")
stacked_labels = labels.stack(sample=("y", "x"))

flat_features = stacked_features.data.astype("float32")
flat_labels = stacked_labels.data.astype("int32")

valid_rows = da.isfinite(flat_labels)
valid_rows &= da.isfinite(flat_features).all(axis=1)

flat_features = flat_features[valid_rows]
flat_labels = flat_labels[valid_rows]

In [8]:
features_np = flat_features.compute()      # NumPy array
labels_np = flat_labels.compute() 

In [9]:
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    features_np,
    labels_np,
    test_size=1 - train_fraction,
    random_state=random_state,
)

In [14]:
X_train_cudf = cudf.DataFrame(X_train_np, columns=band_names)
y_train_cudf = cudf.Series(y_train_np, name="worldcover")
X_test_cudf = cudf.DataFrame(X_test_np, columns=band_names)
y_test_cudf = cudf.Series(y_test_np, name="worldcover")

In [15]:
# %%
X_train_ddf = dask_cudf.from_cudf(X_train_cudf, npartitions=n_workers)
y_train_ddf = dask_cudf.from_cudf(y_train_cudf, npartitions=n_workers)
X_test_ddf = dask_cudf.from_cudf(X_test_cudf, npartitions=n_workers)
y_test_ddf = dask_cudf.from_cudf(y_test_cudf, npartitions=n_workers)


In [16]:
X_train_ddf, y_train_ddf = persist_across_workers(client, [X_train_ddf, y_train_ddf], workers=workers)
X_test_ddf, y_test_ddf = persist_across_workers(client, [X_test_ddf, y_test_ddf], workers=workers)


/raid/jjayabaskar/gis12/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 258.22 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/raid/jjayabaskar/gis12/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 64.56 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [17]:
rf = DaskRandomForest(
    n_estimators=300,
    max_depth=18,
    n_bins=128,
    n_streams=8,
    bootstrap=True,
    split_criterion="gini",
    ignore_empty_partitions=True,
    random_state=random_state,
)


In [18]:
rf.fit(X_train_ddf, y_train_ddf)

In [19]:
cuml_y_pred = rf.predict(X_test_ddf).compute().to_numpy()

/raid/jjayabaskar/gis12/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 1.20 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [23]:
print("CuML accuracy:     ",accuracy_score(y_test_np, cuml_y_pred))

CuML accuracy:      0.799155086995926


In [ ]:
pred_y = rf.predict(X_test_ddf)

pred_cpu = pred_y.compute().to_pandas()
test_cpu = y_test_ddf.compute().to_pandas()

print("\nClass distribution (validation set):")
print(test_cpu.value_counts().sort_index())

print("\nClassification report:")
print(classification_report(test_cpu, pred_cpu, labels=valid_classes))

cm = confusion_matrix(test_cpu, pred_cpu, labels=valid_classes)
cm_df = xr.DataArray(cm, coords={"actual": valid_classes, "predicted": valid_classes},
                     dims=("actual", "predicted"))
cm_df

/raid/jjayabaskar/gis12/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 1.20 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(



Class distribution (validation set):
worldcover
1      203204
2      520680
4        3446
5        3985
7     1588132
8        1455
11      96625
Name: count, dtype: int64

Classification report:


/raid/jjayabaskar/gis12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/raid/jjayabaskar/gis12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/raid/jjayabaskar/gis12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])

              precision    recall  f1-score   support

           1       0.97      0.90      0.93    203204
           2       0.65      0.62      0.63    520680
           4       0.00      0.00      0.00      3446
           5       0.00      0.00      0.00      3985
           7       0.83      0.88      0.85   1588132
           8       0.00      0.00      0.00      1455
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.64      0.26      0.37     96625

    accuracy                           0.80   2417527
   macro avg       0.34      0.30      0.31   2417527
weighted avg       0.79      0.80      0.79   2417527



<xarray.DataArray (actual: 9, predicted: 9)> Size: 648B
array([[ 183621,    2637,       0,       0,   16776,       0,       0,
              0,     170],
       [   1436,  322245,       0,       0,  195631,       0,       0,
              0,    1368],
       [    107,      72,       0,       0,    3065,       0,       0,
              0,     202],
       [      3,      28,       0,       0,    3717,       0,       0,
              0,     237],
       [   4724,  170446,       0,       0, 1400649,       0,       0,
              0,   12313],
       [      0,       0,       0,       0,    1455,       0,       0,
              0,       0],
       [      0,       0,       0,       0,       0,       0,       0,
              0,       0],
       [      0,       0,       0,       0,       0,       0,       0,
              0,       0],
       [    137,    3915,       0,       0,   67109,       0,       0,
              0,   25464]])
Coordinates:
  * actual     (actual) int64 72B 1 2 4 5 7 8 9 10 11
  * predicted  (predicted) int64 72B 1 2 4 5 7 8 9 10 11